In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv('/kaggle/input/liverpool-ion-switching/train.csv')
test = pd.read_csv('/kaggle/input/liverpool-ion-switching/test.csv')
train.head()

In [ ]:
train.shape, test.shape

In [ ]:
from sklearn.model_selection import train_test_split
X = train.drop(['open_channels','time'], axis=1)
y = train['open_channels']
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.15, shuffle=True)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
model = DecisionTreeClassifier()
model.fit(x_train, y_train)

In [ ]:
from lightgbm import LGBMClassifier
model = LGBMClassifier(boosting_type='gbdt',
                       learning_rate=0.08,
                       min_child_weight=0.01,
                       colsample_bytree=0.5,
                       random_state=2020)

model.fit(x_train,y_train,
          eval_set=[(x_train,y_train),(x_val, y_val.values)],
          early_stopping_rounds=100,
          verbose=200)

pred_y = model.predict(x_val)

In [ ]:
model.score(x_train, y_train)

In [ ]:
pred_val = model.predict(x_val)
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

f1_score(y_val, pred_val, average='weighted'), accuracy_score(y_val, pred_val)

In [ ]:
pd.DataFrame(confusion_matrix(y_val, pred_val))

In [ ]:
pred_test = model.predict(test)

In [ ]:
submission = pd.DataFrame()
submission['time'] = test['time']
submission['open_channels'] = pred_test
submission['open_channels'].value_counts()

In [ ]:
submission.to_csv('DTC.csv', index=False, header=True)